### Lab 8.1 Tokenization

In this lab you will explore different methods of text tokenization.  

The following code will download the text of Shakespeare's sonnets and read it in as one long string.

In [1]:
from torch.utils.data import Dataset

In [2]:
!wget --no-clobber "https://www.dropbox.com/scl/fi/7r68l64ijemidyb9lf80q/sonnets.txt?rlkey=udb47coatr2zbrk31hsfbr22y&dl=1" -O sonnets.txt
text = (open("sonnets.txt").read())


File ‘sonnets.txt’ already there; not retrieving.


In [3]:
text = text.lower()

In [4]:
print(text[:1000])

﻿i

 from fairest creatures we desire increase,
 that thereby beauty's rose might never die,
 but as the riper should by time decease,
 his tender heir might bear his memory:
 but thou, contracted to thine own bright eyes,
 feed'st thy light's flame with self-substantial fuel,
 making a famine where abundance lies,
 thy self thy foe, to thy sweet self too cruel:
 thou that art now the world's fresh ornament,
 and only herald to the gaudy spring,
 within thine own bud buriest thy content,
 and tender churl mak'st waste in niggarding:
   pity the world, or else this glutton be,
   to eat the world's due, by the grave and thee.

 ii

 when forty winters shall besiege thy brow,
 and dig deep trenches in thy beauty's field,
 thy youth's proud livery so gazed on now,
 will be a tatter'd weed of small worth held:
 then being asked, where all thy beauty lies,
 where all the treasure of thy lusty days;
 to say, within thine own deep sunken eyes,
 were an all-eating shame, and thriftless praise.

### Exercises

1. Prepare a vocabulary of the unique words in the dataset.  (For simplicity's sake you can leave the punctuation in.)

In [5]:
vocab = set(text.split(' '))
len(vocab)

4942

2. Now you will make a Dataset subclass that can return sequences of tokens, encoded as integers.

In [6]:
class WordDataset(Dataset):
    def __init__(self, text, seq_len=100):
        self.seq_len = seq_len
        # add code to compute the vocabulary (copied from exercise 1)
        words = text.split(' ')
        self.vocab = [word for word in set(words)]
        # add code to convert the text to a sequence of word indices
        encode = {word: idx for idx, word in enumerate(self.vocab)}
        self.sequence = [encode[word] for word in words if word in encode]
    
    def __len__(self):
        # return the number of possible sub-sequences 
        return len(self.sequence) * self.seq_len

    def __getitem__(self, i):
        # return the sequence of token indices starting at i and the index of token i+seq_len as the label
        return self.sequence[i:i+self.seq_len], self.sequence[i+self.seq_len]

    def decode(self, tokens):
        # convert a sequence of tokens back into a string 
        return ' '.join([self.vocab[token] for token in tokens])

3. Verify that your class can successfully encode and decode sequences.

In [7]:
text_to_encode = '''when forty winters shall besiege thy brow,
 and dig deep trenches in thy beauty's field,
 thy youth's proud livery so gazed on now,
 will be a tatter'd weed of small worth held:
 then being asked, where all thy beauty lies,
 where all the treasure of thy lusty days;
 to say, within thine own deep sunken eyes,
 were an all-eating shame, and thriftless praise.'''

seq_len = 4
dataset = WordDataset(text_to_encode, seq_len=seq_len)
len(dataset)

252

In [8]:
dataset[5]

([29, 34, 48, 12], 47)

In [9]:
dataset.decode(dataset.sequence)

"when forty winters shall besiege thy brow,\n and dig deep trenches in thy beauty's field,\n thy youth's proud livery so gazed on now,\n will be a tatter'd weed of small worth held:\n then being asked, where all thy beauty lies,\n where all the treasure of thy lusty days;\n to say, within thine own deep sunken eyes,\n were an all-eating shame, and thriftless praise."

4. Do the exercise again, but this time at the character level.

In [10]:
class CharacterDataset(Dataset):
    def __init__(self, text, seq_len=100):
        self.seq_len = seq_len
        # add code to compute the vocabulary of unique characters
        letters = [letter for letter in text]
        self.vocab = [letter for letter in set(letters)]
        # add code to convert the text to a sequence of character indices
        encode = {letter: idx for idx, letter in enumerate(self.vocab)}
        self.sequence = [encode[letter] for letter in letters if letter in encode]
        
    def __len__(self):
        # return the number of possible sub-sequences
        return len(self.sequence) * self.seq_len
    def __getitem__(self,i):
        # return the sequence of token indices starting at i and the index of token i+1 as the label
        return self.sequence[i:i+self.seq_len], self.sequence[i+self.seq_len]

    def decode(self,tokens):
        # convert a sequence of tokens back into a string
        return ''.join([self.vocab[token] for token in tokens])

In [11]:
text_to_encode = '''when forty winters shall besiege thy brow,
 and dig deep trenches in thy beauty's field,
 thy youth's proud livery so gazed on now,
 will be a tatter'd weed of small worth held:
 then being asked, where all thy beauty lies,
 where all the treasure of thy lusty days;
 to say, within thine own deep sunken eyes,
 were an all-eating shame, and thriftless praise.'''

dataset2 = CharacterDataset(text_to_encode, seq_len=seq_len)
len(dataset2)

1440

In [12]:
len(dataset2)

1440

In [13]:
dataset2[5]

([10, 30, 8, 27], 0)

In [14]:
dataset2.decode(dataset2.sequence)

"when forty winters shall besiege thy brow,\n and dig deep trenches in thy beauty's field,\n thy youth's proud livery so gazed on now,\n will be a tatter'd weed of small worth held:\n then being asked, where all thy beauty lies,\n where all the treasure of thy lusty days;\n to say, within thine own deep sunken eyes,\n were an all-eating shame, and thriftless praise."

5. Compare the number of sequences for each tokenization method.

In [15]:
print(f'Using sequence length: {seq_len}')
print(f'Word tokenization: {len(dataset)}')
print(f'Letter tokenization: {len(dataset2)}')

Using sequence length: 4
Word tokenization: 252
Letter tokenization: 1440


Letter tokenization is much larget

6. Optional: implement the byte pair encoding algorithm to make a Dataset class that uses word parts.

I'm sorry Dr. Ventura it's been a very busy quarter